In [1]:
# data: any image contains a background, a workpiece and pattern on it

# framework: sklearn.cluster
# model: kmeans clustering
    # objective: extract only "pattern" to .png transparent
    # method: read step by step below

# result: work on all test images

from sklearn.cluster import KMeans
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# input

raw_image = cv.imread("images/image1.png")
resized_image = cv.resize(raw_image, (0, 0), fx=0.5, fy=0.5)
realimage = cv.cvtColor(resized_image, cv.COLOR_BGR2RGB)
plt.imshow(realimage)

In [ ]:
# gray, blur, canny edge

gray = cv.cvtColor(resized_image, cv.COLOR_BGR2GRAY)
blurred = cv.GaussianBlur(gray, (5, 5), 0)
edges = cv.Canny(blurred, 50, 150)

plt.imshow(edges)

In [ ]:
# extract largest contours

contours, _ = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
contours = sorted(contours, key=cv.contourArea, reverse=True)
largest_contour = np.zeros_like(gray)
cv.drawContours(largest_contour, [contours[0]], -1, 255, thickness=cv.FILLED)

plt.imshow(largest_contour)

In [ ]:
# and: take inner path of image
# or: make part that black (outter part) colored, pattern similar to black, also changed to almost-colored

blue_background = np.zeros_like(resized_image)
blue_background[:, :] = [0, 0, 255]
image_with_contour = cv.bitwise_and(resized_image, resized_image, mask=largest_contour)
image_or = cv.bitwise_or(blue_background, image_with_contour)

plt.imshow(image_or)

In [ ]:
# and: take inner path of previous step (colored pattern now)
# xor: background exact same color: removed. inner same white: black. pattern change to a completly different color

image_with_contour = cv.bitwise_and(image_or, image_or, mask=largest_contour)
image_xor = cv.bitwise_xor(blue_background, image_with_contour)

plt.imshow(image_xor)

In [ ]:
# clustering into clusters

data = image_xor.reshape(-1, 3)
kmeans = KMeans(n_clusters=3, random_state=0).fit(data)
clustered_image = kmeans.labels_.reshape(image_xor.shape[:2])

plt.imshow(clustered_image)

In [ ]:
# extract smallest cluster

clustered_data = np.bincount(clustered_image.flatten())
pattern_index = np.argmin(clustered_data)
pattern = (clustered_image == pattern_index).astype(np.uint8)

plt.imshow(pattern)

In [ ]:
# split a cluster into connected components

nums, connected_patterns, stats, centroids = cv.connectedComponentsWithStats(pattern, connectivity=8)

plt.imshow(connected_patterns)

In [ ]:
# remove small size (noise) connected components

pattern_area = clustered_data[pattern_index]
threshold_value = 0.05
threshold = int(threshold_value * pattern_area)

clean_pattern = np.zeros_like(pattern)
for i in range(1, nums):
    if stats[i, cv.CC_STAT_AREA] > threshold:
        clean_pattern[connected_patterns == i] = 255

plt.imshow(clean_pattern)

In [ ]:
# fill background "transparent"

clean_pattern_height, clean_pattern_width = clean_pattern.shape
image_transpattern = np.zeros((clean_pattern_height, clean_pattern_width, 4), dtype=np.uint8)
image_transpattern[clean_pattern != 0] = [0, 0, 0, 255]
image_transpattern[clean_pattern == 0] = [0, 0, 0, 0]

plt.imshow(image_transpattern)

In [ ]:
# save result

cv.imwrite('image_transpattern.png', image_transpattern)